In [1]:
import numpy as np
import scipy.stats
import msprime
import pandas as pd
import allel
import multiprocessing
import pyarrow # required for pandas Data.Frame.to_arrow()

# Functions

### Demographic simulations

In [2]:
def sim_population(diploid_size, split_time, seq_len, rec_rate, mut_rate):
    '''simulate two populations that have diverged
        from a common ancestral population.
    Returns a tree sequence.

    @diploid_size = the population size of each population,
        also the size of the ancestral population
    @split_time = current populations split from the
        ancestral population this many generations ago
    @seq_len = length of the genome, units ~ base-pairs
    @rec_rate = recombination rate, units = rate per bp, per generation
    @mut_rate = mutation rate, units = rate per bp, per generation
    '''

    demography = msprime.Demography()
    demography.add_population(name="A", initial_size=diploid_size)
    demography.add_population(name="B", initial_size=diploid_size)
    demography.add_population(name="C", initial_size=diploid_size)
    demography.add_population_split(time=split_time,
                                    derived=["A", "B"], ancestral="C")

    ts = msprime.sim_ancestry(
        samples={'A': diploid_size, 'B': diploid_size},  # diploid samples
        demography=demography,
        ploidy=2,
        sequence_length=seq_len,
        discrete_genome=False,
        recombination_rate=rec_rate,
        model='dtwf',
    )

    ts = msprime.sim_mutations(
        ts,
        rate=mut_rate,
        discrete_genome=False,
        start_time=split_time,
        )

    return ts

In [3]:
def observe(ts, num_inds, max_sites):
    '''
    oberserve num_inds diploids from each population
    simplify the ts, removing non-variable sites across those individuals

    @ts = tree-sequence
    @num_inds = number of diploids to sample from each population
    @max_sites = retain at most max_sites, from among variable sites
    '''

    popA = ts.samples(population=0)
    popB = ts.samples(population=1)
    popA_inds = sample_individuals(popA, num_inds, replace=False)
    popB_inds = sample_individuals(popB, num_inds, replace=False)
    all_inds = np.concatenate([popA_inds, popB_inds])
    obs_ts = ts.simplify(samples=all_inds)

    if obs_ts.num_sites > max_sites:
        all_sites = np.arange(len(obs_ts.sites()))
        sites_keep = np.random.choice(all_sites, max_sites, replace=False)
        sites_remove = np.setdiff1d(all_sites, sites_keep)
        obs_ts = obs_ts.delete_sites(sites_remove)
    return(obs_ts)

In [4]:
def sample_individuals(haploid_indexes, n, replace):
    """
    return the (haploid) indexes that correspond to
    taking n diploid samples from the supplied haploid indexes

    @haploid_indexes = contiguous indexes for haploids
        should be from a single population.
    @n = the number of diploid indiviudals to take.
    """

    # ensure the haploid indexes are consecutive
    diff = np.diff(haploid_indexes)
    assert np.sum(diff == 1) == (len(haploid_indexes)-1)

    ind_indexes = haploid_indexes[::2]
    ind_samples = np.sort(np.random.choice(ind_indexes, n, replace=replace))
    haploid_samples = np.zeros(len(ind_samples)*2, dtype='int')
    haploid_samples[0::2] = ind_samples
    haploid_samples[1::2] = ind_samples+1
    return(haploid_samples)

# Example simulation and observation

In [5]:
# simulation of the entire demographic history
pop_ts = sim_population(
    diploid_size=200,
    split_time=50,
    seq_len=1e9,
    rec_rate=1e-8,
    mut_rate=1e-8
)

 sim_population() generates two populations that have diverged from a common ancestral population. 
 
 Here we have specified:
 
 `diploid_size = 200` so each population is composed of 200 diploid individuals.
 
 `split_time = 50` so the two populations split from each other 50 generations ago.
 
 `seq_len = 1e9` so the genome is ~ a billion basepairs (bp) long.
 
 `rec_rate = 1e-8` so the recombination rate is .00000001 per bp per generation.
 
 `mut_rate = 1e-8` so the mutation rate is .00000001 per bp per generation (same as the mut_rate).

## Data structure
The genetic data is store as a tree sequence [link to documentation](https://tskit.dev/tskit/docs/stable/data-model.html)

In [6]:
# for a tree-sequence, "samples" are always haploid,
# while we will generally be addressing diploid individuals
pop_ts.num_samples

800

This ts has 800 "samples". This is due to having two populations, of 200 diploids each. 

`200 (diploids)* 2 (pops) * 2 (samples per diploid)`

### The `ts` above contains the full history of all indiviudals in both populations. 

In practice, researchers rarely get to observe all individuals or variable sites.  

#### We can also simulate a situation where we only get to observe a subset of the individuals/sites.

In [7]:
# The observe() function returns another ts that contains
#     a subset of the individuals and/or sites.
obs_ts = observe(ts=pop_ts, num_inds=50, max_sites=5000)

 Here we have specified:
 
 `ts = pop_ts` so the pop_ts is the initial 2-population ts, with 200 individuals per pop.
 
 `num_inds = 50` so there will be 50 inds individuals per pop in the returned ts.
 
 `max_sites = 5000` so their will be at most 5000 variable sites in the returned ts.

In [8]:
# 50 individuals * 2 (pops) * 2 (samples per diploid)
obs_ts.num_samples

200

In [9]:
# the number of sites has also be reduced
obs_ts.num_sites, pop_ts.num_sites

(5000, 31508)

# Summary statistics

In [10]:
def get_fst_general(ts, popA_samples, popB_samples, sites_index):
    """returns Hudson's Fst

    This function is general in the sense that all of:
      (popA_samples, popB_samples, and sites_index) may have duplicates.

    @ts = tree sequence
    @popA_samples = the samples from the first population to be used
    @popB_samples = the samples from the second population to be used
    @sites_index = the indexes of the sites to be used.
    """

    ga = allel.GenotypeArray(
        ts.genotype_matrix().reshape(
            ts.num_sites, ts.num_samples, 1),
        dtype='i1')
    # count alleles within each population at the selected sites and inds
    ac1 = ga[sites_index][:, popA_samples, :].count_alleles()
    ac2 = ga[sites_index][:, popB_samples, :].count_alleles()
    # calculate Hudson's Fst (weighted)
    num, denom = allel.hudson_fst(ac1, ac2)
    fst = np.sum(num) / np.sum(denom)
    return fst

In [11]:
def get_fst_mult(params):
    '''
    wrapper around get_fst_general() that takes the params as a single tuple.
    For use with multiprocessing pool.imap()
    '''
    ts, popA_samples, popB_samples, sites_resample = params
    return get_fst_general(ts, popA_samples, popB_samples, sites_resample)

In [12]:
def get_target(summary_func, ts):
    '''
    apply summary_func to all sites and samples in a tree-sequence

    @summary_func = the function to be applied
    @ts = tree-sequence
    '''

    target = summary_func(
        ts=ts,
        popA_samples=ts.samples(population=0),
        popB_samples=ts.samples(population=1),
        sites_index=np.arange(len(ts.sites()))
    )
    return target

### We use summary statistics to summarize the  complex demographic history in a simple way

Fst is summary statistic that is a measure of population differentiation due to genetic structure. 

Put one way, how similar are a pair of indiviudals from the same population, relative to a pair of indiviudals from different populations?

\begin{equation}
F_{ST}   = \frac{\pi_{between} - \pi_{within}}{\pi_{between}}
\end{equation} 

$ \pi_{between} $ = the average number of pairwise differences between two samples from different populations.

$ \pi_{within} $  = the average number of pairwise differences between two samples from the same population.


Here we use Hudson's Fst estimator (https://doi.org/10.1101/gr.154831.113).

Fst is calculated by the function `get_fst_general()`.
The function `get_target()` applies this function using all the individuals and variable sites in a specified ts. 

In [13]:
# Fst across all individuals and sites
get_target(summary_func=get_fst_general, ts=pop_ts)

0.11282235492034863

In [14]:
# across the *observed* individuals and sites
get_target(summary_func=get_fst_general, ts=obs_ts)

0.10897508807239904

In [15]:
# if we observe a different set of individuals and sites,
# there is a different Fst
obs2_ts = observe(ts=pop_ts, num_inds=50, max_sites=5000)
get_target(summary_func=get_fst_general, ts=obs2_ts)

0.1155999260579587

In [16]:
# Similarly, if we simulate a new pair of populations,
# they will have a different Fst
pop2_ts = sim_population(
    diploid_size=200,
    split_time=50,
    seq_len=1e9,
    rec_rate=1e-8,
    mut_rate=1e-8
)
get_target(summary_func=get_fst_general, ts=pop2_ts)

0.11022474466868462

###  We can also consider the expected value of Fst, based on how we have set up the simulation. 
Briefly, the `diploid_size` and `split_time` parameters determine the expected value of Fst (and most other summary stats we may consider).

While the `seq_len`, `rec_rate`, and `mut_rate` parameters affect how much variance there is *between* simulation runs.  Higher values of these three parameters will tend to reduce the variance across runs.

For a citation see equation s5 in (https://doi.org/10.1101/gr.154831.113).

In [17]:
def get_expected_Fst(N, T):
    '''
    Expected Fst between populations that have split T generations ago,
        without ongoing migration.
    Assumes the population size of each population is the same.

    citation: eq s5 from Bhatia et al. (https://doi.org/10.1101/gr.154831.113)

    @N = diploid population size of each population
    @T = split time, in generations
    '''
    return 1 - (1-(1/(2*N)))**T

In [18]:
# for our current case with N = 200 and T = 50
get_expected_Fst(N=200, T=50)

0.11764120699699332

Notice for an `observed` subset, the expected Fst does not change, as the relevant N is the size of the population.

# Experimental design

Often, researchers have data for a subset of individuals / sites from a study system of one or more populations.

Usually, researchers are interested in learning about **populations**, rather than just the subset of indiviudals they observe.

One question of interest is how closely summary stats based on the observed subset match summary stats from the entire population. This will depend on the demographic history of the system (e.g. population size), aspects of study design (e.g. number of observed individuals and sites), as well as statistical aspects of the particular summary stat (e.g. what aspects of the data the summary stat is sensitive to). 

Another, related question is if resampling methods applied to observed subsets can be used to accurately quantify the uncertainty in population-level summary statistics. Wow well do jackknife/bootstrap-based confidence intervals (CIs) perform? For example, does a 95% CI contain the population-level values 95% of the time?

In this project our focus on this last issue.

# Resampling methods
The functions below calculate Fst across resampled sets of individuals and sites. 

In [19]:
def bootstrap_ind_fst(ts, n_boot, n_cores):
    '''
    Calculate Fst while bootstrap resampling over individuals.
    uses multiprocessing.Pool to run across multiple cores.

    @ts = tree-sequence
    @n_boot = number of times to resample with replacement
    @n_cores = number of multiprocessing cores to use
    '''

    popA = ts.samples(population=0)
    popB = ts.samples(population=1)
    sites_index = np.arange(len(ts.sites()))

    bt_vals = []
    pool = multiprocessing.Pool(n_cores)
    popA_nind = int(len(popA)/2)
    popB_nind = int(len(popB)/2)

    inputs = [(ts,
               sample_individuals(popA, popA_nind, replace=True),
               sample_individuals(popB, popB_nind, replace=True),
               sites_index
               ) for _ in range(n_boot)]
    for res in pool.imap(get_fst_mult, inputs):
        bt_vals.append(res)

    return bt_vals

In [20]:
def jackknife_ind_fst(ts, n_cores):
    '''
    Calculate Fst while jackknife resampling over individuals.
    uses multiprocessing.Pool to run across multiple cores

    @ts = tree-sequence
    @n_cores = number of multiprocessing cores to use
    '''

    popA = ts.samples(population=0)
    popB = ts.samples(population=1)
    sites_index = np.arange(ts.num_sites)

    jk_vals = []  # Fst while leaving each individual out
    pool = multiprocessing.Pool(n_cores)

    popA_nind = int(len(popA)/2)
    popB_nind = int(len(popB)/2)

    inputs = [(ts,
               sample_individuals(popA, popA_nind, replace=True),
               popB,
               sites_index
               ) for _ in range(popA_nind)]
    inputs += [(ts,
               popA,
               sample_individuals(popB, popB_nind, replace=True),
               sites_index
                ) for _ in range(popB_nind)]

    for res in pool.imap(get_fst_mult, inputs):
        jk_vals.append(res)

    return jk_vals

In [21]:
def resample_inds_fst(ts, n_boot, n_cores):
    '''
    Calculates Fst for bootstrap and jackknife resampling of individuals.

    @ts = tree-sequence
    @n_boot = number of bootstrap re-samplings
    @n_cores = number of multiprocessing cores for individual-based bootstrap
    '''

    res = {}

    # Fst bootstrap
    bt_vals = bootstrap_ind_fst(
        ts=ts,
        n_boot=n_boot,
        n_cores=n_cores
    )

    # Fst jackknife
    jk_vals = jackknife_ind_fst(
        ts=ts,
        n_cores=n_cores
    )

    res['bt_inds'] = bt_vals
    res['jk_inds'] = jk_vals
    # additional resampling methods could be used,
    # just add them here and include a new name in the res dictionary
    # with the value set to the Fst across the resampled data sets.

    return res

In [22]:
def resample_sites_fst(ts, n_boot):
    '''
    Calculate Fst for bootstrap and jackknife resampling of sites.

    @ts = tree-sequence
    @n_boot = number of bootstrap re-samplings
    '''

    # ts saves populations with indexes starting at 0
    popA = ts.samples(population=0)
    popB = ts.samples(population=1)
    res = {}

    # get Fst num and denom at each site
    ga = allel.GenotypeArray(
        ts.genotype_matrix().reshape(
            ts.num_sites, ts.num_samples, 1),
        dtype='i1')
    # count alleles within each population at the selected sites and inds
    ac1 = ga[:, popA, :].count_alleles()
    ac2 = ga[:, popB, :].count_alleles()
    # calculate Hudson's Fst (weighted)
    # num, denome are the per-site of the numerator and denominator
    num, denom = allel.hudson_fst(ac1, ac2)

    # do bootstrap, sample sites with replacement, n_boot times
    # bt_weights is a 2D-array giving the number of times
    # each site is sampled for each of the n_boot replicates
    bt_weights = np.random.multinomial(
        n=ts.num_sites,
        pvals=np.ones(ts.num_sites)/ts.num_sites,
        size=n_boot
    )
    # weight the per-site values of (num, denom) by bt_weights, sum over reps
    bt_vals = (num*bt_weights).sum(1) / (denom*bt_weights).sum(1)

    # jackknife - rather than generating the weighting via random sampling,
    # we exclude each site in turn
    numsum = num.sum()
    denomsum = denom.sum()
    jk_vals = (numsum - num) / (denomsum - denom)

    res['bt_sites'] = bt_vals
    res['jk_sites'] = jk_vals
    # additional resampling methods could be used, (e.g. block jacknife)
    # just add them here and include a new name in the res dictionary
    # with the value set to the Fst across the resampled data sets.

    return res

## Here we investigate how the number of individuals and sites observed affects the coverage of CIs generated via resampling.

In each case, we observe [ninds = 25 or 50] diploid individuals and [nsites = 1000, or 100000] sites. 

We conduct 5 base simulations, and for each combination of **ninds** and **nsites** we conduct jackknife and bootstrap resampling 20 times [nrep] over both sites and individuals. Each simulation has population with size 200 and a split time of 50 generations. 

For each bootstrap replicate, we conduct 100 [n_boot] samplings with replacement. For each jackknife replicate, we leave each item out once.

We return the results in a [pandas.DataFrame](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html), with some metadata about each analysis, as well as the Fst calculated on each resampling iteration. 

In [23]:
n_cores = 10
n_boot = 100
res = [] # list to hold results

nsim = 5
ninds = [25, 50]
nsites = [1000, 100000]
nrep = 20
for S in range(nsim):
    pop_ts = sim_population(
        diploid_size=200, 
        split_time=50,
        seq_len=1e9,
        rec_rate=1e-8,
        mut_rate=1e-8
    )
    sim_fst = get_expected_Fst(N=200, T=50) # N, T should match values passed to sim_population()
    pop_fst = get_target(summary_func=get_fst_general, ts=pop_ts)
    for N in ninds:
        for L in nsites:
            for rep in range(nrep): 
                obs_ts = observe(pop_ts, num_inds=N, max_sites=L)
                obs_fst = get_target(summary_func=get_fst_general, ts=obs_ts)
                # resample over inds
                res_ind = resample_inds_fst(ts=obs_ts, n_boot=n_boot, n_cores=n_cores)
                for (method, vals) in res_ind.items():
                    res.append([S, N, L, rep, obs_ts.num_sites, sim_fst, pop_fst, obs_fst, method, vals])
                
                # resample over sites
                res_site = resample_sites_fst(ts=obs_ts, n_boot=n_boot)
                # calculate confidence intervals here
                for (method, vals) in res_site.items():
                    res.append([S, N, L, rep, obs_ts.num_sites, sim_fst, pop_fst, obs_fst, method, vals])
            print(f'finished: {S}, {N}, {L}')

finished: 0, 25, 1000
finished: 0, 25, 100000
finished: 0, 50, 1000
finished: 0, 50, 100000
finished: 1, 25, 1000
finished: 1, 25, 100000
finished: 1, 50, 1000
finished: 1, 50, 100000
finished: 2, 25, 1000
finished: 2, 25, 100000
finished: 2, 50, 1000
finished: 2, 50, 100000
finished: 3, 25, 1000
finished: 3, 25, 100000
finished: 3, 50, 1000
finished: 3, 50, 100000
finished: 4, 25, 1000
finished: 4, 25, 100000
finished: 4, 50, 1000
finished: 4, 50, 100000


In [24]:
res_df = pd.DataFrame(res)
res_df.columns = ['simID', 'N_obs', 'L_obs', 'rep_obs', 'nsites', 'sim_fst', 'pop_fst', 'obs_fst', 'method', 'vals']
res_df

,simID,N_obs,L_obs,rep_obs,nsites,sim_fst,pop_fst,obs_fst,method,vals
0,0,25,1000,0,1000,0.117641,0.110806,0.105560,bt_inds,"[0.12775023751921952, 0.11950721922013961, 0.1..."
1,0,25,1000,0,1000,0.117641,0.110806,0.105560,jk_inds,"[0.12007719735488832, 0.11109491650488501, 0.1..."
2,0,25,1000,0,1000,0.117641,0.110806,0.105560,bt_sites,"[0.11222246565315662, 0.10589863191019262, 0.1..."
3,0,25,1000,0,1000,0.117641,0.110806,0.105560,jk_sites,"[0.10557292821519125, 0.10575149851493383, 0.1..."
4,0,25,1000,1,1000,0.117641,0.110806,0.103995,bt_inds,"[0.12137080991078142, 0.13247973713263858, 0.1..."
...,...,...,...,...,...,...,...,...,...,...
1595,4,50,100000,18,31060,0.117641,0.111999,0.110391,jk_sites,"[0.11039113767279728, 0.11039140373084193, 0.1..."
1596,4,50,100000,19,31034,0.117641,0.111999,0.111737,bt_inds,"[0.11700800746740846, 0.12160664457678791, 0.1..."
1597,4,50,100000,19,31034,0.117641,0.111999,0.111737,jk_inds,"[0.11554338161517841, 0.11433622913291283, 0.1..."
1598,4,50,100000,19,31034,0.117641,0.111999,0.111737,bt_sites,"[0.11162668688485056, 0.11133754655552033, 0.1..."


### Writing the results to disk and loading again
Notice saving in text (csv) format converts the arrays within the vals column into strings.


So here we use the feather format [link](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_feather.html)

In [37]:
# requires 
res_df.to_feather(path='trial_data.july22.feather')

In [40]:
res_df2 = pd.read_feather(path='trial_data.july22.feather')
res_df2

,simID,N_obs,L_obs,rep_obs,nsites,sim_fst,pop_fst,obs_fst,method,vals
0,0,25,1000,0,1000,0.117641,0.110806,0.105560,bt_inds,"[0.12775023751921952, 0.11950721922013961, 0.1..."
1,0,25,1000,0,1000,0.117641,0.110806,0.105560,jk_inds,"[0.12007719735488832, 0.11109491650488501, 0.1..."
2,0,25,1000,0,1000,0.117641,0.110806,0.105560,bt_sites,"[0.11222246565315662, 0.10589863191019262, 0.1..."
3,0,25,1000,0,1000,0.117641,0.110806,0.105560,jk_sites,"[0.10557292821519125, 0.10575149851493383, 0.1..."
4,0,25,1000,1,1000,0.117641,0.110806,0.103995,bt_inds,"[0.12137080991078142, 0.13247973713263858, 0.1..."
...,...,...,...,...,...,...,...,...,...,...
1595,4,50,100000,18,31060,0.117641,0.111999,0.110391,jk_sites,"[0.11039113767279728, 0.11039140373084193, 0.1..."
1596,4,50,100000,19,31034,0.117641,0.111999,0.111737,bt_inds,"[0.11700800746740846, 0.12160664457678791, 0.1..."
1597,4,50,100000,19,31034,0.117641,0.111999,0.111737,jk_inds,"[0.11554338161517841, 0.11433622913291283, 0.1..."
1598,4,50,100000,19,31034,0.117641,0.111999,0.111737,bt_sites,"[0.11162668688485056, 0.11133754655552033, 0.1..."
